In [120]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
# [Default]

def OS_Driver(os,browser):
    if os.lower() == 'windows':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS('C:/Users/pc/Documents/phantomjs-2.1.1-window/bin/phantomjs.exe')
        else:
            driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    elif os.lower() == 'mac':
        if browser.lowser() == 'phantom':
            driver = webdriver.PhantomJS(
                '/Users/hyunyoun/Documents/GitHub/Private_Project/phantomjs-2.1.1/bin/phantomjs')
        else:
            driver = webdriver.Chrome('../chromedriver')
    return driver
def OpenAPI(apiFile):
    apiKey = pickle.load(open(apiFile, 'rb'))
    return apiKey
## Define Error
class NotMatch(Exception):
    def __init__(self, f):
        self.f = f
    def __str__(self):
        return self.f
## Setting BeautifulSoup
def Setting_BeautifulSoup(parser):
    if parser.lower() == 'xml':
        parser = 'lxml'
    elif parser.lower() == 'html':
        parser = 'html.parser'
    else:
        raise NotMatch('Not Match parser')
    return parser
## Using BeautifulSoup
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup

In [121]:
def FilterMainText(contents):
    #mainText = list(filter(lambda x: x.PREFIX !='<!--' and x.PREFIX !=None, contents))
    #mainText = list(map(lambda x: x.strip(), mainText))
    #mainText = ''.join(mainText)
    
    mainText = contents.text.replace(u'\xa0', u'')
    return mainText
'''
def FilterMainText(contents):
    mainText = contents.text
    #body = re.sub('(<span.*?>.*?</span>)', '', str(contents),1)
    #mainText = re.sub('<.+?>', '', body, 1)
    mainText = mainText.replace(u'\xa0', u'')
    #mainText = mainText.split('<br/><br/>')
    #mainText = list(map(lambda x: re.sub('<a.*?>.*?</a>', '', x), mainText))
    #mainText = list(filter(lambda x: re.search('(<.+?>)', x) == None, mainText))
    #mainText = list(filter(lambda x: x != '', mainText))
    #mainText = list(map(lambda x: x.strip(), mainText))
    #mainText = ''.join(mainText)
    return mainText
'''
# Search for News Article & Press
def ArticleInNaver(cat, url):
    articleIdList = ['articeBody', 'newsEndContents', 'articleBodyContents']
    if cat == r'연예':
        mainTextId = 'articeBody';
        pressClass = 'press_logo'
    elif cat == r'스포츠':
        mainTextId = 'newsEndContents';
        pressClass = 'logo'
    else:
        mainTextId = 'articleBodyContents';
        pressClass = 'press_logo'
    webPage = WebPageUsingBS(url, 'xml')
    try:
        a = webPage.find('div', id=mainTextId)
        mainText = FilterMainText(a)
    except:
        articleIdList.remove(mainTextId)
        print (articleIdList)
        try:
            mainText = FilterMainText(webPage.find('div', id=articleIdList[0]))
        except:
                mainText = FilterMainText(webPage.find('div', id=articleIdList[1]))
        else:pass
    else:pass
    print (articleIdList)
    press = webPage.find(class_=pressClass)
    press = press.find('img').attrs['alt']
    return mainText, press

In [223]:
res = requests.get(w3)
soup = BeautifulSoup(res.content, 'html.parser')


In [228]:
for idx in soup:
    print (idx)

HTML


<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1023" name="viewport">
<title>실수냐 고의냐…SKT, 평창올림픽 KT 중계망 훼손 논란 : 네이버 뉴스</title>
<meta content="노컷뉴스 : 네이버뉴스" property="me2:post_tag">
<meta content="노컷뉴스" property="me2:category1"/>
<meta content="기타 기사" property="me2:category2"/>
<meta content="http://imgnews.naver.net/image/origin/079/2017/12/04/3040943.jpg" property="me2:image"/>
<meta content="실수냐 고의냐…SKT, 평창올림픽 KT 중계망 훼손 논란" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;oid=079&amp;aid=0003040943&amp;sid1=001" property="og:url"/>
<meta content="http://imgnews.naver.net/image/origin/079/2017/12/04/3040943.jpg" property="og:image"/>
<meta content='KT 관로에 SKT 광케이블 연결하려다 적발…"관로 비어있어 작업자 오인" [CBS노컷뉴스 김연지 기자]SK텔레콤이 두 달여 앞으로 다가온 평창 동계올림픽 KT 중계망을 훼손한 혐의로 고발당했다. SK텔레콤은 현장' property="og:description"/>
<meta content="노

In [217]:
w = 'http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=214&aid=0000791692&date=20171204&type=1&rankingSectionId=102&rankingSeq=21'

In [218]:
ArticleInNaver('문화',w)

['articeBody', 'newsEndContents', 'articleBodyContents']


("\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\n■ 방송 : MBC 라디오 표준FM 95.9MHz <변창립의 시선집중>(07:30~09:00)■ 진행 : 변창립 앵커■ 대담 : 김영란 서강대 법학전문대학원 석좌교수-김영란법, 생각보다 연착륙 더 잘해…출발 좋다-'3·5·10 조항'은 부득이한 경우의 금액…원칙적 전부 금지-숫자는 중요한 게 아냐…금액 올리든 말든 큰 변화 없어-농어민 고충 정면으로 들어드려야…더 적극적 도움 없을까-농어민 도움 찬성하지만 개정이 큰 효과 있을까 의문-방향 옳다면 전체적 방향 되돌리는 건 안 되지 않나 -법원·검찰 갈등 표면화…판결 후 논쟁은 피할 수 없어☎ 진행자 > 부정청탁금지법, 이른바 김영란법이 시작된 지 약 1년 2개월이라는 시간이 지났습니다. 법안이 도입되는 과정부터 시행 후 지금까지도 참 많은 논란과 파장들이 이어지고 있습니다. 현재 정부에서는 내년 설 전에 식사, 선물, 경조사비 상한액인 3·5·10의 기준을 3·5·5로 개정하고 선물비 가액은 농축수산품에 한해서 10만 원으로 올리는 법 개정안을 추진하고 있습니다. 김영란법 시행으로 경제적인 타격을 입고 있는 농어민들을 보호해야 한다는 이유인데 과연 이분은 이런 논의를 어떻게 바라보고 계실까요. 지금 서강대학교 법학전문대학원의 석좌교수시죠. 김영란 전 국민권익위원장과 얘기 나누도록 하겠습니다. 안녕하십니까?☎ 김영란 > 네, 안녕하세요. 김영란입니다.☎ 진행자 > 제가 교수님이라고 부르는 게 더 편하실는지요?☎ 김영란 > 네, 그러시죠.☎ 진행자 > 요즘 어떻게 지내고 계십니까?☎ 김영란 > 지금 아직도 강의가 안 끝나서 2학기 강의 계속하고 있습니다.☎ 진행자 > 학기말 시험치고 이제 성적내고 그러면 또 연말까지 계속 바빠지시겠네요.☎ 김영란 > 네, 그렇습니다.☎ 진행자 > 지난해에 법 시행 초기에요. 김 교수께서는 이 법이 무사히 연착륙할 수 있을까 염려한다고 말씀

In [133]:
driver = webdriver.Chrome('../../../chromedriver.exe')

In [180]:
w2 = 'http://v.media.daum.net/v/20171203070123858'

In [181]:
driver.get(w2)

In [182]:
try:
    element = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'alex-area')))
except:pass
else:
    loop = True
    while loop:
        try:
            commentsByclass = 'alex_more'
            element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, commentsByclass)))
            more_button = driver.find_element_by_class_name(commentsByclass)
            webdriver.ActionChains(driver).move_to_element(more_button).click(more_button).perform()
            commentElements = driver.find_elements_by_class_name('cmt_info')
        except:
            loop = False

In [185]:
def CommentsInDaum(df):
    df[r'공감'] = re.search('[\d]+', df[r'공감']).group()
    df[r'비공감'] = re.search('[\d]+', df[r'비공감']).group()
    if df['comments'] == '':
        comments = '오직 이모티콘만 있는 댓글입니다.'
    else:
        comments = ' '.join(df['comments'].split('\n'))
    df['comments'] = comments
    return df

In [186]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
comments= soup.select('#alex-area > div > div > div > div.cmt_box > ul.list_comment > li > div.cmt_info')

In [187]:
xx = list(map(lambda x: (x.find('p', class_ = 'desc_txt').text, x.find('button',class_ = '#like').text, x.find('button',class_ = '#dislike').text), comments))
xx = pd.DataFrame(xx)

AttributeError: 'NoneType' object has no attribute 'text'

In [177]:
xx.rename({0:'comments',1:'공감',2:'비공감'}, axis = True, inplace = True)

In [178]:
comment_Df['comments'].apply(lambda x: len(x)).value_counts()

,comments,공감,비공감
0,개는 ? 사람을 공격했으면 안락사 시켜야 하는거 아닌가요?,댓글 찬성하기10172,댓글 비추천하기423
1,어제 ebs 에서 잠시보았는데 \n독일서는 개 기르려면 세금도 내고 보험도 든다하더...,댓글 찬성하기3849,댓글 비추천하기43
2,대체 몇 명이 더 희생 당해야 개금지법을 만들지…\n국개~넘들을 물어 죽이게 해야\...,댓글 찬성하기2613,댓글 비추천하기96
3,자고로 외출시 개는 크던작던 무조건 마스크를 착용시켜야한다.,댓글 찬성하기2441,댓글 비추천하기163
4,요즘 개들 때문에 시끄러운 시대에 \n저렇게 나한테 달려들면 \n개턱주가리를 확 돌...,댓글 찬성하기1897,댓글 비추천하기82


In [179]:
xx.apply(lambda x: CommentsInDaum(x),axis = 1)

,comments,공감,비공감
0,개는 ? 사람을 공격했으면 안락사 시켜야 하는거 아닌가요?,10172,423
1,어제 ebs 에서 잠시보았는데 독일서는 개 기르려면 세금도 내고 보험도 든다하더라...,3849,43
2,대체 몇 명이 더 희생 당해야 개금지법을 만들지… 국개~넘들을 물어 죽이게 해야 할지…,2613,96
3,자고로 외출시 개는 크던작던 무조건 마스크를 착용시켜야한다.,2441,163
4,요즘 개들 때문에 시끄러운 시대에 저렇게 나한테 달려들면 개턱주가리를 확 돌려차...,1897,82
5,구속시켜라,1620,35
6,최시원 개 에 지방으로 갔는지 밝혀라 !,1373,37
7,개주인을 잡아가둬야 관리를 하지. 지들한테나 귀엽지 타인한테는 혐오스런 맹수다.,1316,35
8,개를 묶어 놔야지 저리 풀어 노니 사람들이 물리지 주인 늠은 개풀어놓고 어디...,1269,64
9,자꾸 이러면....나가린데..,1060,28


In [189]:
for i in comments:
    try:
        i.find('p',class_='desc_txt').text
        i.find('button',class_ = '#like').text
        i.find('button',class_ = '#dislike').text
    except:
        print (i)
        break

<div class="cmt_info" data-reactid=".0.0.0.4.1.$194532715.0"><strong class="tit_nick" data-reactid=".0.0.0.4.1.$194532715.0.0"><span class="info_author" data-reactid=".0.0.0.4.1.$194532715.0.0.0"><span class="link_nick" data-reactid=".0.0.0.4.1.$194532715.0.0.0.0"><span class="img_daum" data-reactid=".0.0.0.4.1.$194532715.0.0.0.0.0"></span></span><a class="link_nick" data-reactid=".0.0.0.4.1.$194532715.0.0.0.1">탈퇴한 사용자</a><span class="txt_date" data-reactid=".0.0.0.4.1.$194532715.0.0.0.2">2017.12.03 15:33</span></span></strong><p class="desc_txt font_size_17" data-reactid=".0.0.0.4.1.$194532715.0.2"><em class="emph_txt" data-reactid=".0.0.0.4.1.$194532715.0.2.0">삭제</em><span data-reactid=".0.0.0.4.1.$194532715.0.2.1">된 글입니다.</span></p></div>


In [335]:
t2.select_one('p.desc_txt > em.emph_txt') == None

True

,comments
0,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
1,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
2,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
3,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
4,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
5,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
6,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
7,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
8,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."
9,"<div class=""cmt_info"" data-reactid="".0.0.0.4.1..."


In [300]:
len(k)

496

In [294]:
#//*[@id="comment*"]
for idx in enumerate(soup.find_all('p',class_='desc_txt')):
    print (idx[0], idx[1].text)

0 삼가고인의  명복을  빕니다.  구조하시느라 고생하신 분들의  안전도  생각하시길...
1 해경도 다칠라요
조심하세요
2 겨울에 낚시배는 위험한짓이요

안전불감증들
3 제발 무사히 살아서 구조되시길 간절히 바랍니다 !! 그리고 구조하는해경관계자분들깨서도 아무런 사고가 생기지않고 임무수행하시기바랍니다 !!
4 갑작스런 사고에  충격이 큽니다  ㅡ구조요원 여러분고생많으시고  ㅡ삼가 고인의 명복을빕니다
5 이 강추위에 그래도 생존자가 있다는건 구조가 그만큼 빨랐던것 같네요 돌아가신분들에겐 삼가고인의 명복을 비며 생존자분들의 빠른 쾌유를 빕니다
6 일베충들이 설치네 세월호때는 아예 구조 시도도 안했다는 팩트지
7 실종자 두명은 사망한걸로 봐야 한다
겨울바다에서 오래 못 버틴다
8 급유선 운전사 얘기는 어디에도 없네...
9 기사똑바로 써라 전원  구조 라고 8명사망. 2명실종 도무지 이러고도 월급주나
10 삼가 고인에 명복을빔니다. 신속히 구조에 수고하신분들 머리숙여 감사올립니다...
11 비내리고 바람부니 바다 나가지마라는데.왜 기어나가나? 한심한것들..
12 비오고 파도 거센데 조심하셔야지 왜 나가셔가지고,,,,  많이도 돌아가셨내
13 인근인데
해경도착 시간 넘 늦은거 아닌가??
왜??
구명조끼를 입었는데도
불구하고 이렇게 사망자가 많지??
14 삼가 고인의 명복을 빕니다.
이것도 인재인데 누굴 탓할까?
죽은 사람만 억울할 뿐....해경의 대처가 어떠했는지  밝혀볼 여지가 있다.
세월호도 해경의 안일한 대처와 언론의 확인되지 않는 기사로 인명피해를 키웠는데 이번 사고도 그 전철을 그대로 밟았다. 사람이 죽어도 변화지 않는 시스템....얼마나 더 희생되어야 바뀔까?
15 지금 사람이 20명죽어도 어떤 방송사도 사건을 내보내지 않고있다
16 미친 떠라이 박사모 일베충 자유항문당 알바새기
17 세월호와 뭐가 달라
문가도 흙탕물 뒤집어 쓰고
하야 하겠군
18 제앙아  구관이 명관이다  .. 니도 그리 잘할것 같앗나..  지난 10개월 니는 뭘했나,,
일자리 창출이 

In [221]:
w3 = 'http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=079&aid=0003040943&date=20171204&type=1&rankingSectionId=105&rankingSeq=17'

In [160]:
res = requests.get(w)
soup = BeautifulSoup(res.content, 'html.parser')
res2 = requests.get(w2)
soup2 = BeautifulSoup(res2.content, 'html.parser')
res3 = requests.get(w3)
soup3 = BeautifulSoup(res3.content, 'html.parser')

In [174]:
rs = soup.find_all('div', id = 'articleBodyContents')
rs2 = soup2.find_all('div', id = 'articeBody')
rs3 = soup3.find_all('div', id = 'articleBodyContents')

In [175]:
import nltk

In [176]:
rs

[<div id="articleBodyContents">
 <!-- 본문 내용 -->
 <!-- TV플레이어 -->
 <!-- // TV플레이어 -->
 <script type="text/javascript">
 // flash 오류를 우회하기 위한 함수 추가
 function _flash_removeCallback() {}
 </script>
 <p>
 <img height="110" src="http://imgnews.naver.net/image/214/2017/12/04/112742247_title.JPG?type=w540" style="width: 523px; height: 193px;" width="120"/></p><p>■ 방송 : MBC 라디오 표준FM 95.9MHz &lt;변창립의 시선집중&gt;(07:30~09:00)<br/>■ 진행 : 변창립 앵커<br/>■ 대담 : 김영란 서강대 법학전문대학원 석좌교수</p><p><strong>-김영란법, 생각보다 연착륙 더 잘해…출발 좋다<br/>-'3·5·10 조항'은 부득이한 경우의 금액…원칙적 전부 금지<br/>-숫자는 중요한 게 아냐…금액 올리든 말든 큰 변화 없어<br/>-농어민 고충 정면으로 들어드려야…더 적극적 도움 없을까<br/>-농어민 도움 찬성하지만 개정이 큰 효과 있을까 의문<br/>-방향 옳다면 전체적 방향 되돌리는 건 안 되지 않나 <br/>-법원·검찰 갈등 표면화…판결 후 논쟁은 피할 수 없어</strong></p><p>☎ 진행자 &gt; 부정청탁금지법, 이른바 김영란법이 시작된 지 약 1년 2개월이라는 시간이 지났습니다. 법안이 도입되는 과정부터 시행 후 지금까지도 참 많은 논란과 파장들이 이어지고 있습니다. 현재 정부에서는 내년 설 전에 식사, 선물, 경조사비 상한액인 3·5·10의 기준을 3·5·5로 개정하고 선물비 가액은 농축수산품에 한해서 10만 원으로 올리는 법 개정안을 추진하고 있습니다. 김영란법 시행으로 경제적인 타격을 입고 있는 농어민들을 보호해야 한다는 이유인데 

In [177]:
rs2

[<div class="article_body font1 size3" id="articeBody">
 <!-- [D] 본문 폰트 설정 class 안내
 					나눔고딕 : font1
 					맑은고딕 : font2
 					돋움 : font3
 					바탕 : font4
 					폰트 사이즈 1 : size1
 					폰트 사이즈 2 : size2
 					폰트 사이즈 3 : size3
 					폰트 사이즈 4 : size4
 					폰트 사이즈 5 : size5
 				-->
 <b>[COVER STORY] <br/>세계를 사로잡은 '방탄소년단'… 일곱 소년 모으고, 키워낸 방시혁 단독 인터뷰<br/>"요즘 아이돌은 해외파가 필수? 우리 애들은 非서울 토종인걸요"<br/><br/>방시혁이 탄생시킨 소년단?<br/>하하, 그런 뜻은 절대 아냐 청춘의 고통·압박감 <br/>막아주겠단 의도였는데 유치하단 말도 좀 들었죠<br/><br/>전세계 '청춘'과 코드 맞춰<br/>즐겁고 행복한 노래보단 젊은이들의 가혹한 현실<br/>가사에 담으려고 노력 이게 해외서도 먹혔네요<br/><br/>내 정체성은 '상업작곡가'<br/>대중의 취향에 딱 맞는 트렌디한 음악이 내 목표<br/>영감은 어디서 얻냐고요? 음악은 엉덩이로 쓰는 것<br/></b><br/><br/>2010년 경기 일산 대진고 1학년생 김남준군은 '꾼'들 사이에서 소문난 래퍼 지망생이었다. 중학생 때부터 힙합 음악에 빠져 직접 만든 랩과 음악을 힙합 커뮤니티 사이트에 올렸다. 선수들 사이에 알음알음 소문나더니 당시 최고 대중가요 작곡가의 귀에도 이 얘기가 들어갔다.<br/><br/>지오디 '하늘색 풍선', 비의 '나쁜 남자', 백지영 '총 맞은 것처럼' 등 히트곡을 만든 방시혁(45·빅히트 엔터테인먼트 대표)이었다. 김군의 랩을 듣고 단박에 '물건'임을 직감한 방시혁은 자신이 운영하는 기획사 연습생으로 김군을 스카우트했다. 김군을 시작으로 이후 6명의 소년을 추가로 발탁했다. 일산, 과천, 부

In [193]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

In [194]:
data = soup3.findAll(text=True)

In [196]:
list(filter(visible, data))

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 ' Skip navigation ',
 '\n',
 '\n',
 '메인 메뉴로 바로가기',
 '\n',
 '서브 메뉴로 바로가기',
 '\n',
 '본문으로 바로가기',
 '\n',
 '\n',
 ' //Skip navigation ',
 '\n',
 '\n',
 '\n',
 '\n',
 ' GNB 마크업 ',
 '\n',
 '\n',
 ' //GNB 마크업 ',
 '\n',
 '\n',
 '\n',
 'NAVER',
 '\n',
 '뉴스',
 ' ',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '뉴스홈',
 ' ',
 '\n',
 '속보',
 ' ',
 '\n',
 '정치',
 ' ',
 '\n',
 '경제',
 ' ',
 '\n',
 '사회',
 ' ',
 '\n',
 '생활/문화',
 ' ',
 '\n',
 '세계',
 ' ',
 '\n',
 'IT/과학',
 ' ',
 '\n',
 '오피니언',
 ' ',
 '\n',
 '포토',
 ' ',
 '\n',
 'TV',
 ' ',
 '\n',
 '랭킹뉴스',
 ' ',
 '선택됨',
 '\n',
 '\n',
 '\n',
 '\n',
 '뉴스 검색',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '검색',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '12',
 '.',
 '08',

In [55]:
def ExtractComments(class1, class2):
    try:
        commentText = class1.find(class_='u_cbox_contents').text
        recomm = class2.find(class_='u_cbox_btn_recomm').select_one('em').text
        unrecomm = class2.find(class_='u_cbox_btn_unrecomm').select_one('em').text
    except:
        pass
    return pd.DataFrame({'comments': commentText, r'공감': recomm, r'비공감': unrecomm})

In [56]:
def ExtractElementFromRow(row):
    row['comments'] = row['comments'].text
    row['공감'] = row['공감'].select_one('em').text
    row['비공감'] = row['비공감'].select_one('em').text
    return row

In [57]:
def ArticleInNaver(cat, url):
    if cat == r'연예':
        mainTextId = 'articeBody';
        pressClass = 'press_logo'
    elif cat == r'스포츠':
        mainTextId = 'newsEndContents';
        pressClass = 'logo'
    else:
        mainTextId = 'articleBodyContents';
        pressClass = 'press_logo'
    webPage = WebPageUsingBS(url, 'xml')
    mainText = webPage.find('div', id=mainTextId)
    mainText = FilterMainText(mainText)
    press = webPage.find(class_=pressClass)
    press = press.find('img').attrs['alt']
    return mainText, press
def FilterMainText(contents):
    mainText = contents.text
    #body = re.sub('(<span.*?>.*?</span>)', '', str(contents),1)
    #mainText = re.sub('<.+?>', '', body, 1)
    mainText = mainText.replace(u'\xa0', u'')
    #mainText = mainText.split('<br/><br/>')
    #mainText = list(map(lambda x: re.sub('<a.*?>.*?</a>', '', x), mainText))
    #mainText = list(filter(lambda x: re.search('(<.+?>)', x) == None, mainText))
    #mainText = list(filter(lambda x: x != '', mainText))
    #mainText = list(map(lambda x: x.strip(), mainText))
    #mainText = ''.join(mainText)
    return mainText

In [58]:
cat='연예'

In [59]:
article = ArticleInNaver(cat, url)
if cat == r'연예':
    commentByClass = 'reply_count';
    commentNumByClass = 'u_cbox_count'
elif cat == r'스포츠':
    commentByClass = 'comment';
    commentNumByClass = 'u_cbox_count'
else:
    commentByClass = 'pi_btn_count';
    commentNumByClass = 'u_cbox_count'
commentMore = 'u_cbox_paginate'
driver.get(url)
element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, commentByClass)))
moreCommentPage = driver.find_element_by_class_name(commentByClass)
webdriver.ActionChains(driver).move_to_element(moreCommentPage).click(moreCommentPage).perform()
try: 
    element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, commentNumByClass)))
    commentNum = driver.find_element_by_class_name(commentNumByClass).text
    commentNum = ''.join(commentNum.split(','))
except NoSuchElementException:
    element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, commentNumByClass)))
    commentNum = driver.find_element_by_class_name(commentNumByClass).text
    commentNum = ''.join(commentNum.split(','))
else:pass
print('naver Information Number of comment : {}'.format(commentNum))
print('naver Start : Click More Button')
loop = True
while loop == True:
    try:
        element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, commentMore)))
        moreComment = driver.find_element_by_class_name(commentMore)
    except TimeoutException:
        try:
            element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, commentMore)))
            moreComment = driver.find_element_by_class_name(commentMore)
            webdriver.ActionChains(driver).move_to_element(moreComment).click(moreComment).perform()
        except NoSuchElementException:
            loop = False
        except TimeoutException:
            loop = False
    except StaleElementReferenceException:
        try:
            element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, commentMore)))
            moreComment = driver.find_element_by_class_name(commentMore)
            webdriver.ActionChains(driver).move_to_element(moreComment).click(moreComment).perform()
        except StaleElementReferenceException:
            loop = False
        except TimeoutException:
            loop = False
    else:
        webdriver.ActionChains(driver).move_to_element(moreComment).click(moreComment).perform()
        if moreComment.get_attribute('style') != '':
            loop = False
print('naver End Click More Button & Start Crawling comments')
soup = BeautifulSoup(driver.page_source, 'html.parser')
commentsDf = pd.DataFrame({'comments': soup.select('#cbox_module > div > div.u_cbox_content_wrap > ul > li > div.u_cbox_comment_box > div > div.u_cbox_text_wrap > span'),
                               '공감': soup.select('#cbox_module > div > div.u_cbox_content_wrap > ul > li > div.u_cbox_comment_box > div > div.u_cbox_tool > div > a.u_cbox_btn_recomm'),
                               '비공감': soup.select('#cbox_module > div > div.u_cbox_content_wrap > ul > li > div.u_cbox_comment_box > div > div.u_cbox_tool > div > a.u_cbox_btn_unrecomm')
                               })
driver.quit()
commentsDf = commentsDf.apply(lambda x: ExtractElementFromRow(x), axis=1)
print('naver Number of comment : {}'.format(len(commentsDf)))
print ('naver End')


naver Information Number of comment : 397
naver Start : Click More Button
naver End Click More Button & Start Crawling comments
naver Number of comment : 397
naver End


In [60]:
from konlpy.tag import *
hannanum = Hannanum()
kkma = Kkma()
twitter = Twitter()

In [117]:
import numpy as np
import matplotlib.pylab as plt

In [118]:
from sklearn.feature_extraction.text import CountVectorizer

In [119]:
mor = twitter.morphs(article[0])

In [120]:
pos = twitter.pos(article[0])

In [121]:
vect = CountVectorizer().fit(mor)

In [3]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial

In [98]:
u = 'http://v.sports.media.daum.net/v/20171202102023226'

In [99]:
def CommentsInDaum(df):
    recomm =  df['comments'].find('span', class_='comment_recomm')
    recomm = list(map(lambda x: x.text, recomm.find_all('button')))
    recomm = list(map(lambda x: re.search('[\d]+', x).group(), recomm))
    df[r'공감'] = recomm[0]
    df[r'비공감'] = recomm[1]
    if df['comments'].find('p').text == '':
        comments = '이미지 또는 특수문자입니다. '
    else:
        comments = '\s'.join(df['comments'].find('p').text.split('\n'))
    df['comments'] = comments
    return df

In [100]:
driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')

In [108]:
driver.get(u)

In [153]:
# -*- coding: utf-8 -*-
import pickle
import urllib3
import json
def OpenAPI():
    import pickle
    openapi = pickle.load(open('./etri_open_api_access_key', 'rb'))
    return openapi
def USE_ETRI_AI(analysisCode, text):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    accessKey = OpenAPI()
    requestJson = {
        'access_key' : accessKey,
        'argument' : {
            'text' : text,
            'analysis_code' : analysisCode
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    return response
if __name__ == "__main__":
    text = """경구용 피임약은 여성호르몬을 강제로 조정해 배란을 억제하는 원리다. 호르몬 변화로 인해 신체에 다양한 증상이 함께 나타난다. 유방암과 우울증 위험을 높인다는 연구결과가 있는 반면, 류마티스 관절염 위험을 낮춘다는 연구결과도 있다.
최근에는 피임약 복용이 이성의 취향까지 바꾼다는 연구결과가 발표됐다. 이성을 선택할 때 선호하는 얼굴을 바꾼다는 내용의 연구다. 영국 스코틀랜드 스털링대학 연구진은 피임약을 복용하는 만18~24세 여성 18명과 복용하지 않는 37명을 대상으로 각각 2회에 걸쳐 취향을 조사했다.
조사에 사용된 얼굴은 컴퓨터 그래픽으로 남녀 20명의 사진을 합성해서 만들었다. 모니터에 매력적이라고 생각하는 얼굴이 나타나면 선택하도록 했다. 첫 번째 조사는 두 그룹 모두 피임약을 복용하지 않을 때 진행했다. 그 중에서도 임신 가능성이 가장 높은 배란 1~2일 전에 조사가 이뤄졌다. 두 번째 조사 역시 배란기에 해당하는 시기에 이뤄졌다. 다만, 한 그룹은 피임약을 복용하는 상태였다.
그 결과, 피임약을 복용하지 않은 그룹은 두 차례의 조사에서 모두 남성적인 얼굴에서 매력을 느끼는 것으로 조사됐다. 반면, 피임약을 복용한 그룹의 경우 복용 전에는 남성적인 얼굴에 매력을 느끼다가 복용 후에는 이 비율이 매우 낮아졌다.
    """
    a = USE_ETRI_AI('morp',text)

In [154]:
a.status

200

In [105]:
comment_Df = soup.select('#alex-area > div > div > div > div.cmt_box > ul.list_comment > li')
comment_Df = pd.DataFrame(comment_Df)
comment_Df.rename({0:'comments'}, axis = 1,inplace = True)
comment_Df = comment_Df.apply(lambda x: CommentsInDaum(x), axis = 1)

In [106]:
comment_Df

,comments,공감,비공감
0,한국 18.3프로??\s생각보다 높네,1088,40
1,빨리 탈락하고 정상 생활 하자. 밤 새지마라...,785,46
2,한국은 제로다..협회부터 물갈이하지않는이상..호구다..,211,13
3,16강진출이아니라..1무2패냐 3패냐가 문제라고생각햇는데..그럼 18.3프로 이해가...,167,7
4,신태용이 있는 한 국대 응원할 맘 전혀 없다,181,25
5,18% 라고 ㅋㅋㅋ\s0% 다\s괜히 기대감 갖지마세요~,151,5
6,아프리카와 아시아는 그냥 들러리네,135,6
7,다 천적느낌인데 3전 전패 예상한다 ㅋ,105,7
8,사우디나 튀니지 보단 높네 그나마 ㅋㅋㅋ,72,5
9,"끝에서 세번째 확율 ?\s그냥 끝인데 ,",69,5


In [117]:
x = 'http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003334021&date=20171201&type=1&rankingSectionId=103&rankingSeq=14'

In [118]:
# Filtering News Article
def FilterMainText(contents):
    mainText = list(filter(lambda x: x.PREFIX !='<!--' and x.PREFIX !=None, contents))
    mainText = list(map(lambda x: x.strip(), mainText))
    mainText = ''.join(mainText)
    mainText = mainText.replace(u'\xa0', u'')
    return mainText
# Search for News Article & Press
def ArticleInNaver(cat, url):
    articleIdList = ['articeBody', 'newsEndContents', 'articleBodyContents']
    if cat == r'연예':
        mainTextId = 'articeBody';
        pressClass = 'press_logo'
    elif cat == r'스포츠':
        mainTextId = 'newsEndContents';
        pressClass = 'logo'
    else:
        mainTextId = 'articleBodyContents';
        pressClass = 'press_logo'
    webPage = WebPageUsingBS(url, 'xml')
    try:
        mainText = FilterMainText(webPage.find('div', id=mainTextId))
    except AttributeError:
        articleIdList.remove(mainTextId)
        try:
            mainText = FilterMainText(webPage.find('div', id=articleIdList[0]))
        except AttributeError:
                mainText = FilterMainText(webPage.find('div', id=articleIdList[1]))
        else:pass
    else:pass
    press = webPage.find(class_=pressClass)
    press = press.find('img').attrs['alt']
    return mainText, press

In [119]:
ArticleInNaver('문화',x)

('크리스마스트리가 사라졌다. 겨울이면 흔하디흔했던 전통 크리스마스트리가 좀처럼 보이지 않는 요즘 크리스마스 인테리어 쇼핑 성지인 ‘고터’(서울고속버스터미널 지하상가), 남대문시장에도 올해는 크리스마스트리 판매하는 곳이 눈에 띄게 줄었다. 예년 같으면 11월 초, 늦어도 중순이면 크리스마스 인테리어로 단장했을 매장들도 11월 말이 되어서야 하나둘 옷을 갈아입는 분위기다.인테리어·라이프스타일 전문 매장도 마찬가지다. ‘모던하우스’ ‘자주(JAJU)’ ‘이케아’ 등도 전통 트리 품목이 대폭 줄어든 대신 단순하면서도 소박한 느낌을 살린 크리스마스 인테리어 장식품이 주를 이룬다. 김지훈 이케아 PR·미디어 매니저는 “요즘은 소비 심리 위축, 단순하고 깔끔한 미니멀 인테리어 유행으로 크고 웅장한 전통 크리스마스트리보단 부담 없이 겨울 분위기를 연출할 수 있는 캔들, 조명 등 트리 대용 장식품이 인기”라고 했다.크리스마스트리는 갈수록 간소해지고 있다. 몇 년 전부터 거실 한쪽에 세우는 거대한 트리 대신 벽에 거는 형태의 ‘행잉 트리’와 ‘캔버스 트리’가 등장하더니 지난해부턴 트리를 그려 넣은 현수막 형태의 ‘패브릭 트리’가 유행이다. 블로그, 인스타그램 등엔 냉장고와 벽에 접착식 메모지인 포스트잇(post-it)으로 트리 형태를 만들어 붙인 ‘포스트잇 트리’, 크리스마스 무늬 포장지로 트리 모양을 만들어 붙인 ‘포장지 트리’나 ‘스티커 트리’ 등도 심심찮게 올라온다. 모두 큰돈 들이지 않고 크리스마스 분위기 내며 자기만족을 느끼는 ‘가심비(가격 대비 심리적 만족도)’ 큰 트리들. 하지만 자칫 조악해지기 쉽다. 4명의 스타일링 전문가에게 최소한의 재료로 ‘가심비 트리’ 만드는 노하우를 들어봤다.벽에 거는 \'행잉 트리\', 캔버스 위에 나뭇가지를 붙여 만든 \'캔버스 트리\'를 유행시킨 플로리스트 이재희(36) 딜라이트 대표는 "이번 크리스마스에도 여전히 벽걸이 형태의 크리스마스 장식품이 유행할 것"이라고 했다. "자연친화적인 북유럽 인테리어가 계속 강세여서 크리스마스 장

In [ ]:
#comment > div > div

In [97]:
soup.select('div > div > span.comment_recomm')

[<span class="comment_recomm" data-reactid=".0.0.0.4.1.$194243553.0.7.1"><button class="btn_g btn_recomm #like ?c_title=%EB%8C%93%EA%B8%80%EC%B0%AC%EC%84%B1" data-reactid=".0.0.0.4.1.$194243553.0.7.1.0"><span class="img_cmt ico_recomm " data-reactid=".0.0.0.4.1.$194243553.0.7.1.0.0">댓글 찬성하기</span><span class="num_txt" data-reactid=".0.0.0.4.1.$194243553.0.7.1.0.1">1247</span></button><button class="btn_g btn_oppose #dislike ?c_title=%EB%8C%93%EA%B8%80%EB%B0%98%EB%8C%80" data-reactid=".0.0.0.4.1.$194243553.0.7.1.1"><span class="img_cmt ico_oppose " data-reactid=".0.0.0.4.1.$194243553.0.7.1.1.0">댓글 비추천하기</span><span class="num_txt" data-reactid=".0.0.0.4.1.$194243553.0.7.1.1.1">135</span></button></span>,
 <span class="comment_recomm" data-reactid=".0.0.0.4.1.$194243925.0.7.1"><button class="btn_g btn_recomm #like ?c_title=%EB%8C%93%EA%B8%80%EC%B0%AC%EC%84%B1" data-reactid=".0.0.0.4.1.$194243925.0.7.1.0"><span class="img_cmt ico_recomm " data-reactid=".0.0.0.4.1.$194243925.0.7.1.0.0">댓글 

In [18]:
x = datetime.now()
x

datetime.datetime(2017, 12, 6, 0, 19, 42, 154935)

In [20]:
y = x.date() - timedelta(days=1)

In [21]:
y

datetime.date(2017, 12, 5)

In [49]:
y.strftime("%Y%m%d")

'20171205'

In [54]:
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup
# --------------------------------------------------------------------
# Crawling for Naver
# --------------------------------------------------------------------
# Resource for Naver
def Resource_Naver():
    apiInfo = OpenAPI('./naverClient_Info.json')
    basePage = 'http://news.naver.com'
    targetPage = 'http://news.naver.com/main/ranking/popularDay.nhn?'
    return apiInfo, basePage, targetPage
# Search for Date
def SearchDateForNaver(mainpage, url):
    webpage = WebPageUsingBS(url, 'xml')
    dateInfo = webpage.find('div', class_='calendar_date')
    #today = dateInfo.find(class_='c_date').text
    yesterdayPage = dateInfo.find('a').attrs['href']
    targetPage = mainpage + yesterdayPage
    webpage2 = WebPageUsingBS(targetPage, 'xml')
    dateInfo2 = webpage2.find('div', class_='calendar_date')
    #targetDate = dateInfo2.find(class_='c_date').text
    return targetDate, targetPage
# Search for category Name & web path
def SearchTargetForNaver(date, mainpage, form):
    category = re.search(r'[a-zA-Z가-힣/]+', form.text).group()
    if category == r'연예':
        targetDay = '-'.join(date.split('.'))[:-1]
        link = mainpage + form.find('a').attrs['href'] + '#type=hit_total&date=' + targetDay
    else:
        link = link = mainpage + form.find('a').attrs['href']
    return {'category': category, 'link': link}
# Search for category web path
def CategoryWebPathForNaver(date, mainpage, url):
    webPage = WebPageUsingBS(url, 'xml')
    category = webPage.find('ul', class_='massmedia').find_all('li')[1:-2]
    outDf = pd.DataFrame(list(map(lambda x: SearchTargetForNaver(date, mainpage, x), category)))
    return outDf


In [55]:
basePage = 'http://news.naver.com'
targetPage = 'http://news.naver.com/main/ranking/popularDay.nhn?'

In [56]:
SearchDateForNaver(basePage, targetPage)

('2017.12.04.',
 'http://news.naver.com/main/ranking/popularDay.nhn;jsessionid=48013810F7C082AC3EFC29C2A6037852?rankingType=popular_day&date=20171204')

In [57]:
webpage = WebPageUsingBS(targetPage, 'xml')

In [60]:
dateInfo = webpage.find('div', class_='calendar_date')
dateInfo

<div class="calendar_date">
<a class="btn pre" href="/main/ranking/popularDay.nhn;jsessionid=1AA3721C3324249B7FBDF3EB59771868?rankingType=popular_day&amp;date=20171204" title="이전"><span>이전</span></a>
<span class="c_date">2017.12.05.</span>
<a class="btn next_off" href="#" onclick="return false;" title="다음"><span>다음</span></a>
<div class="cal_area">
<a class="btn cal" href="#" id="u.c.button" title="달력보기"><span>달력보기</span></a>
<div class="layer01 layer03" id="u.c.layer">
<div class="calendar layer01_content">
<div class="calendar_inner">
<div class="head">
<div>
<div class="skin01">
<select id="u.c.year.select" style="display:none;" title="년도 선택"></select>
<div class="selectbox" id="u.c.year.div">
<a class="box" href="#">
<span class="label"></span>
<span class="button"><span class="blind">연도선택</span></span>
</a>
<div class="list scrollbox" style="overflow: hidden; z-index: 999;">
<div class="content" style="overflow: hidden; margin-right: 0px;">
<ul style="margin: 0pt; padding: 0pt; po

In [77]:
target_Date = datetime.now()

In [78]:
targetDate = target_Date - timedelta(days=2)

In [83]:
target_Date.date().strftime('%Y.%m.%d')

'2017.12.06'

In [59]:
yesterdayPage = dateInfo.find('a').attrs['href']
yesterdayPage

'/main/ranking/popularDay.nhn;jsessionid=1AA3721C3324249B7FBDF3EB59771868?rankingType=popular_day&date=20171204'

In [80]:
webpage = WebPageUsingBS(targetPage, 'xml')
dateInfo = webpage.find('div', class_='calendar_date')
linkInfo = dateInfo.select_one('a.btn.now')['href']
targetPage = basePage + linkInfo

targetPage

'http://news.naver.com/main/ranking/popularDay.nhn;jsessionid=7808580F77F80D228B7FB43A605802DD?rankingType=popular_day&date=20171204'

In [84]:
soup = WebPageUsingBS('http://media.daum.net/ranking/popular/', 'xml')
date = soup.find(class_='box_calendar')
todayIs = date.find(class_='screen_out').text
compileIs = re.compile('[\d]+')
todayDate = '.'.join(compileIs.findall(todayIs))
linkFortargetDate = soup.select_one('a.btn_date.btn_prev').attrs['href']


In [85]:
linkFortargetDate

'/ranking/popular/?regDate=20171205'